In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action="ignore")

In [2]:
data=pd.read_excel(r"C:\Users\User\Desktop\Machine Learning Project\Biodiesel-Yield-Using-ANN-main\Xantham_gum_data.xlsx")
data.head()

,X1,X2,X3,Observed
0,1,2,7.5,7.22
1,3,2,7.5,7.52
2,1,6,7.5,4.58
3,3,6,7.5,6.45
4,1,4,0.0,4.47


In [3]:
def get_preds(y_test, y_preds):
    y_test=pd.DataFrame(y_test)
    y_test.rename(columns={0:'Actual'}, inplace=True)
    y_preds=pd.DataFrame(y_preds)
    y_preds.rename(columns={0:'Predicted'}, inplace=True)
    predictions=pd.concat([y_test, y_preds], axis=1)
    return predictions    

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from math import sqrt
from sklearn.metrics import r2_score, mean_absolute_error,mean_squared_error

In [5]:
target="Observed"
X=data.drop(columns=[target])
y=data[target]

In [6]:
X1=X.values
y1=y.values.reshape(-1,1)
X_train, X_test, y_train, y_test= train_test_split(X1, y1, test_size=0.3, random_state=100)
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)
x_df=sc.transform(X1)
sc_y=StandardScaler()
y_train=sc_y.fit_transform(y_train)
y_test=sc_y.transform(y_test)
y_df=sc_y.transform(y1)

In [7]:
from scipy.linalg import pinv2, inv
import time

class elm():
    '''
    Function: elm class init
    -------------------
    Parameters:
    shape: list, shape[hidden units, output units]
        numbers of hidden units and output units
    activation_function: str, 'sigmoid', 'relu', 'sin', 'tanh' or 'leaky_relu'
        Activation function of neurals
    x: array, shape[samples, features]
        train data
    y: array, shape[samples, ]
        labels
    C: float
        regularization parameter
    elm_type: str, 'clf' or 'reg'
        'clf' means ELM solve classification problems, 'reg' means ELM solve regression problems.
    one_hot: bool, Ture or False, default True 
        The parameter is useful only when elm_type == 'clf'. If the labels need to transformed to
        one_hot, this parameter is set to be True
    random_type: str, 'uniform' or 'normal', default:'normal'
        Weight initialization method
    '''
    def __init__(self, hidden_units, activation_function,  x, y, C, elm_type, one_hot=True, random_type='normal'):
        self.hidden_units = hidden_units
        self.activation_function = activation_function
        self.random_type = random_type
        self.x = x
        self.y = y
        self.C = C
        self.class_num = np.unique(self.y).shape[0]     
        self.beta = np.zeros((self.hidden_units, self.class_num))   
        self.elm_type = elm_type
        self.one_hot = one_hot

        # if classification problem and one_hot == True
        if elm_type == 'clf' and self.one_hot:
            self.one_hot_label = np.zeros((self.y.shape[0], self.class_num))
            for i in range(self.y.shape[0]):
                self.one_hot_label[i, int(self.y[i])] = 1

        # Randomly generate the weight matrix and bias vector from input to hidden layer
        # 'uniform': uniform distribution
        # 'normal': normal distribution
        if self.random_type == 'uniform':
            self.W = np.random.uniform(low=0, high=1, size=(self.hidden_units, self.x.shape[1]))
            self.b = np.random.uniform(low=0, high=1, size=(self.hidden_units, 1))
        if self.random_type == 'normal':
            self.W = np.random.normal(loc=0, scale=0.5, size=(self.hidden_units, self.x.shape[1]))
            self.b = np.random.normal(loc=0, scale=0.5, size=(self.hidden_units, 1))

    # compute the output of hidden layer according to different activation function
    def __input2hidden(self, x):
        self.temH = np.dot(self.W, x.T) + self.b

        if self.activation_function == 'sigmoid':
            self.H = 1/(1 + np.exp(- self.temH))

        if self.activation_function == 'relu':
            self.H = self.temH * (self.temH > 0)

        if self.activation_function == 'sin':
            self.H = np.sin(self.temH)

        if self.activation_function == 'tanh':
            self.H = (np.exp(self.temH) - np.exp(-self.temH))/(np.exp(self.temH) + np.exp(-self.temH))

        if self.activation_function == 'leaky_relu':
            self.H = np.maximum(0, self.temH) + 0.1 * np.minimum(0, self.temH)

        return self.H

    # compute the output
    def __hidden2output(self, H):
        self.output = np.dot(H.T, self.beta)
        return self.output

    '''
    Function: Train the model, compute beta matrix, the weight matrix from hidden layer to output layer
    ------------------
    Parameter:
    algorithm: str, 'no_re', 'solution1' or 'solution2'
        The algorithm to compute beta matrix
    ------------------
    Return:
    beta: array
        the weight matrix from hidden layer to output layer
    train_score: float
        the accuracy or RMSE
    train_time: str
        time of computing beta
    '''
    def fit(self, algorithm):
        self.time1 = time.time()   # compute running time
        self.H = self.__input2hidden(self.x)
        if self.elm_type == 'clf':
            if self.one_hot:
                self.y_temp = self.one_hot_label
            else:
                self.y_temp = self.y
        if self.elm_type == 'reg':
            self.y_temp = self.y
        # no regularization
        if algorithm == 'no_re':
            self.beta = np.dot(pinv2(self.H.T), self.y_temp)
        # faster algorithm 1
        if algorithm == 'solution1':
            self.tmp1 = inv(np.eye(self.H.shape[0])/self.C + np.dot(self.H, self.H.T))
            self.tmp2 = np.dot(self.tmp1, self.H)
            self.beta = np.dot(self.tmp2, self.y_temp)
        # faster algorithm 2
        if algorithm == 'solution2':
            self.tmp1 = inv(np.eye(self.H.shape[0])/self.C + np.dot(self.H, self.H.T))
            self.tmp2 = np.dot(self.H.T, self.tmp1)
            self.beta = np.dot(self.tmp2.T, self.y_temp)
        self.time2 = time.time()

        # compute the results
        self.result = self.__hidden2output(self.H)
        # If the problem if classification problem, the output is softmax
        if self.elm_type == 'clf':
            self.result = np.exp(self.result)/np.sum(np.exp(self.result), axis=1).reshape(-1, 1)

        # Evaluate training results
        # If problem is classification, compute the accuracy
        # If problem is regression, compute the RMSE
        if self.elm_type == 'clf':
            self.y_ = np.where(self.result == np.max(self.result, axis=1).reshape(-1, 1))[1]
            self.correct = 0
            for i in range(self.y.shape[0]):
                if self.y_[i] == self.y[i]:
                    self.correct += 1
            self.train_score = self.correct/self.y.shape[0]
        if self.elm_type == 'reg':
            self.train_score = np.sqrt(np.sum((self.result - self.y) * (self.result - self.y))/self.y.shape[0])
        train_time = str(self.time2 - self.time1)
        return self.beta, self.train_score, train_time

    '''
    Function: compute the result given data
    ---------------
    Parameters:
    x: array, shape[samples, features]
    ---------------
    Return:
    y_: array
        predicted results
    '''
    def predict(self, x):
        self.H = self.__input2hidden(x)
        self.y_ = self.__hidden2output(self.H)
        if self.elm_type == 'clf':
            self.y_ = np.where(self.y_ == np.max(self.y_, axis=1).reshape(-1, 1))[1]

        return self.y_

    '''
    Function: compute accuracy or RMSE given data and labels
    -------------
    Parameters:
    x: array, shape[samples, features]
    y: array, shape[samples, ]
    -------------
    Return:
    test_score: float, accuracy or RMSE
    '''
    def score(self, x, y):
        self.prediction = self.predict(x)
        if self.elm_type == 'clf':
            self.correct = 0
            for i in range(y.shape[0]):
                if self.prediction[i] == y[i]:
                    self.correct += 1
            self.test_score = self.correct/y.shape[0]
        if self.elm_type == 'reg':
            self.test_score = np.sqrt(np.sum((self.result - self.y) * (self.result - self.y))/self.y.shape[0])

        return self.test_score



In [8]:
import time
start_time=time.time()
model = elm(hidden_units=50, activation_function='relu', random_type='normal', x=X_train, y=y_train, C=1, elm_type='reg')
elapsed_time=time.time()-start_time
print(f'executed in {elapsed_time} seconds or {elapsed_time*1000} milliseconds')

executed in 0.014794349670410156 seconds or 14.794349670410156 milliseconds


In [79]:
beta, train_score, running_time = model.fit('no_re')
print("regression beta:\n", beta)
print("regression train score:", train_score)
print("regression running time:", running_time)

regression beta:
 [[ 4.91630768e-02]
 [-1.27673777e-01]
 [ 8.29182396e-02]
 [-1.24860300e-02]
 [ 8.74626980e-17]
 [-1.38158561e-03]
 [-4.98107229e-03]
 [-5.17462525e-02]
 [ 5.41024437e-02]
 [ 4.85891188e-03]
 [ 5.74083390e-03]
 [ 1.00630337e-01]
 [ 2.00213189e-01]
 [ 7.39240551e-02]
 [-3.48747310e-02]
 [-1.14547440e-01]
 [ 1.02522113e-01]
 [ 4.26288739e-02]
 [-1.56211214e-01]
 [-8.12991751e-02]
 [ 4.23896279e-02]
 [-2.15564962e-02]
 [-2.71447502e-01]
 [-2.12330016e-01]
 [-3.42423519e-02]
 [-1.09150305e-01]
 [ 6.37088802e-03]
 [ 1.04916964e-02]
 [ 7.48924893e-02]
 [ 1.42455343e-01]
 [-1.74669219e-01]
 [ 4.74438793e-02]
 [ 0.00000000e+00]
 [ 1.64900271e-02]
 [-2.05001173e-02]
 [ 1.23115975e-02]
 [-5.70076110e-02]
 [-4.36177640e-02]
 [ 1.39407802e-02]
 [-2.67363407e-02]
 [ 3.38096011e-02]
 [-2.67173551e-02]
 [ 1.75788455e-01]
 [ 1.25102083e-01]
 [ 1.02198580e-01]
 [ 9.40390706e-02]
 [ 5.27694313e-03]
 [ 1.80078595e-01]
 [ 3.04784848e-02]
 [-2.35048591e-01]]
regression train score: 0.07728

In [85]:
prediction_train = model.predict(X_train)
print("regression result:", prediction.reshape(-1, ))
print("regression score:", model.score(X_train, y_train))

regression result: [-0.18779502 -0.56499244 -1.52861396 -1.62586017  1.76891658  0.80529505
 -0.18779502  1.28268553  0.12457159 -0.18779502  0.30138288]
regression score: 0.07728506318568673


In [86]:
y_train1 = sc_y.inverse_transform(y_train)
pred_train = sc_y.inverse_transform(prediction_train.reshape(-1,1))
elm_prediction_train=get_preds(y_train1, pred_train)
elm_prediction_train

,Actual,Predicted
0,6.22,6.096667
1,5.67,5.670000
2,4.58,4.580000
3,4.47,4.470000
4,8.31,8.310000
5,7.22,7.220000
6,6.21,6.096667
7,7.76,7.760000
8,6.45,6.450000
9,5.86,6.096667


In [87]:
corr_train=elm_prediction_train["Actual"].corr(elm_prediction_train["Predicted"])
corr_train

0.9970090365730817

In [88]:
print("TRAIN Mean absolute error :", mean_absolute_error(y_train1,pred_train))
print("TRAIN Mean squared error :",mean_squared_error(y_train1,pred_train))
print("TRAIN Root Mean squared error :",sqrt(mean_squared_error(y_train1,pred_train)))
print("TRAIN R2 Score :",r2_score(y_train1,pred_train))

TRAIN Mean absolute error : 0.04303030303030362
TRAIN Mean squared error : 0.007642424242424222
TRAIN Root Mean squared error : 0.08742095997198968
TRAIN R2 Score : 0.9940270190083844


In [83]:
prediction_test = model.predict(X_test)

In [89]:
y_test1 = sc_y.inverse_transform(y_test)
pred_test = sc_y.inverse_transform(prediction_test.reshape(-1,1))
elm_prediction_test=get_preds(y_test1, pred_test)
elm_prediction_test

,Actual,Predicted
0,6.28,6.096667
1,4.52,4.445082
2,6.93,6.925696
3,7.52,7.727526
4,5.81,6.096667
5,6.46,6.424957


In [90]:
corr_test=elm_prediction_test["Actual"].corr(elm_prediction_test["Predicted"])
corr_test

0.9875684797050625

In [99]:
print("TEST Mean absolute error :", mean_absolute_error(y_test1,pred_test))
print("TEST Mean squared error :",mean_squared_error(y_test1,pred_test))
print("TEST Root Mean squared error :",sqrt(mean_squared_error(y_test1,pred_test)))
print("TEST R2 Score :",r2_score(y_test1,pred_test))

TEST Mean absolute error : 0.1319651150384257
TEST Mean squared error : 0.027619199986825414
TEST Root Mean squared error : 0.16619025238209795
TEST R2 Score : 0.9687726536247749


In [94]:
prediction_df = model.predict(x_df)

In [97]:
y_df1 = sc_y.inverse_transform(y_df)
pred_df = sc_y.inverse_transform(prediction_df.reshape(-1,1))
elm_prediction_df=get_preds(y_df1, pred_df)
elm_prediction_df

,Actual,Predicted
0,7.22,7.220000
1,7.52,7.727526
2,4.58,4.580000
3,6.45,6.450000
4,4.47,4.470000
5,5.67,5.670000
6,6.46,6.424957
7,7.76,7.760000
8,6.65,6.650000
9,4.52,4.445082


In [98]:
corr_df=elm_prediction_df["Actual"].corr(elm_prediction_df["Predicted"])
corr_df

0.9938134841736415

In [100]:
print("FULL DATA Mean absolute error :", mean_absolute_error(y_df1,pred_df))
print("FULL DATA Mean squared error :",mean_squared_error(y_df1,pred_df))
print("FULL DATA Root Mean squared error :",sqrt(mean_squared_error(y_df1,pred_df)))
print("FULL DATA R2 Score :",r2_score(y_df1,pred_df))

FULL DATA Mean absolute error : 0.07441906020964083
FULL DATA Mean squared error : 0.014693050975742292
FULL DATA Root Mean squared error : 0.12121489584924079
FULL DATA R2 Score : 0.9871201939664119


In [120]:
model1 = elm(hidden_units=50, activation_function='leaky_relu', random_type='normal', x=X_train, y=y_train, C=1, elm_type='reg')

In [121]:
beta, train_score, running_time = model1.fit('solution1')
print("regression beta:\n", beta)
print("regression train score:", train_score)
print("regression running time:", running_time)

regression beta:
 [[-0.07582857]
 [ 0.051428  ]
 [ 0.05661829]
 [ 0.04996149]
 [ 0.08860329]
 [ 0.03269863]
 [ 0.12240455]
 [-0.14691729]
 [ 0.12736332]
 [ 0.04802412]
 [-0.038976  ]
 [-0.06407126]
 [-0.00033169]
 [ 0.03727431]
 [ 0.06424671]
 [ 0.05964288]
 [-0.00799503]
 [-0.10088277]
 [ 0.02705199]
 [-0.05710991]
 [ 0.13674625]
 [ 0.10305118]
 [ 0.0623825 ]
 [ 0.08435581]
 [-0.14220799]
 [-0.00695537]
 [ 0.04374524]
 [ 0.02762613]
 [-0.00525959]
 [-0.15737618]
 [-0.02680718]
 [-0.00902521]
 [ 0.00388299]
 [ 0.20060648]
 [ 0.00994542]
 [-0.11702533]
 [-0.01742323]
 [-0.19478933]
 [ 0.15127425]
 [-0.07372018]
 [ 0.00133105]
 [-0.00398348]
 [ 0.19090932]
 [ 0.09264271]
 [-0.0107462 ]
 [-0.01050402]
 [-0.01112077]
 [-0.07656022]
 [ 0.08288803]
 [-0.05995509]]
regression train score: 0.10450585464470323
regression running time: 0.0009984970092773438


In [122]:
prediction_train_leaky = model1.predict(X_train)
y_train_leaky = sc_y.inverse_transform(y_train)
pred_train_sin = sc_y.inverse_transform(prediction_train_leaky.reshape(-1,1))
elm_prediction_train_leaky=get_preds(y_train_leaky, pred_train_sin)
elm_prediction_train_leaky

,Actual,Predicted
0,6.22,6.171967
1,5.67,5.762873
2,4.58,4.637983
3,4.47,4.512423
4,8.31,8.187838
5,7.22,7.162069
6,6.21,6.171967
7,7.76,7.805962
8,6.45,6.327850
9,5.86,6.171967


In [123]:
print("TRAIN leaky Mean absolute error :", mean_absolute_error(y_train,prediction_train_sin))
print("TRAIN leaky Mean squared error :",mean_squared_error(y_train,prediction_train_sin))
print("TRAIN leaky Root Mean squared error :",sqrt(mean_squared_error(y_train,prediction_train_sin)))
print("TRAIN leaky R2 Score :",r2_score(y_train,prediction_train_sin))

TRAIN leaky Mean absolute error : 0.08533579434185366
TRAIN leaky Mean squared error : 0.010539152788092579
TRAIN leaky Root Mean squared error : 0.10266037593975866
TRAIN leaky R2 Score : 0.9894608472119074


In [129]:
pred_test_leaky=model1.predict(X_test)
r2_score(y_test,pred_test_leaky)

0.9213579943132063

In [12]:
model2= elm(hidden_units=50, activation_function='sin', random_type='normal', x=X_train, y=y_train, C=1, elm_type='reg')

In [13]:
beta, train_score, running_time = model2.fit('solution1')

In [14]:
prediction_train_tanh = model2.predict(X_train)
y_train_tanh = sc_y.inverse_transform(y_train)
pred_train_tanh = sc_y.inverse_transform(prediction_train_tanh.reshape(-1,1))
elm_prediction_train_tanh=get_preds(y_train_tanh, pred_train_tanh)
elm_prediction_train_tanh

,Actual,Predicted
0,6.22,6.141355
1,5.67,5.765183
2,4.58,4.647315
3,4.47,4.562987
4,8.31,8.247510
5,7.22,7.083947
6,6.21,6.141355
7,7.76,7.815229
8,6.45,6.213194
9,5.86,6.141355


In [15]:
print("TRAIN tanh Mean absolute error :", mean_absolute_error(y_train,prediction_train_tanh))
print("TRAIN tanh Mean squared error :",mean_squared_error(y_train,prediction_train_tanh))
print("TRAIN tanh Root Mean squared error :",sqrt(mean_squared_error(y_train,prediction_train_tanh)))
print("TRAIN tanh R2 Score :",r2_score(y_train,prediction_train_tanh))

TRAIN tanh Mean absolute error : 0.10068220475735777
TRAIN tanh Mean squared error : 0.01420508084826863
TRAIN tanh Root Mean squared error : 0.11918506973723106
TRAIN tanh R2 Score : 0.9857949191517313


In [16]:
pred_test_tan=model2.predict(X_test)
r2_score(y_test,pred_test_tan)

0.8792203075368237

In [110]:
model3= elm(hidden_units=50, activation_function='sin', random_type='normal', x=X_train, y=y_train, C=1, elm_type='reg')

In [118]:
beta, train_score, running_time = model3.fit('solution2')

In [124]:
prediction_train_sin = model3.predict(X_train)
y_train_sin = sc_y.inverse_transform(y_train)
pred_train_sin = sc_y.inverse_transform(prediction_train_sin.reshape(-1,1))
elm_prediction_train_sin=get_preds(y_train_sin, pred_train_sin)
elm_prediction_train_sin

,Actual,Predicted
0,6.22,6.195549
1,5.67,5.742551
2,4.58,4.547866
3,4.47,4.592756
4,8.31,8.236588
5,7.22,7.059644
6,6.21,6.195549
7,7.76,7.755135
8,6.45,6.226038
9,5.86,6.195549


In [125]:
print("TRAIN sin Mean absolute error :", mean_absolute_error(y_train,prediction_train_sin))
print("TRAIN sin Mean squared error :",mean_squared_error(y_train,prediction_train_sin))
print("TRAIN sin Root Mean squared error :",sqrt(mean_squared_error(y_train,prediction_train_sin)))
print("TRAIN sin R2 Score :",r2_score(y_train,prediction_train_sin))

TRAIN sin Mean absolute error : 0.09442582136092338
TRAIN sin Mean squared error : 0.016216818956722338
TRAIN sin Root Mean squared error : 0.12734527457555045
TRAIN sin R2 Score : 0.9837831810432777


In [127]:
test_pred_sin=model3.predict(X_test)
r2_score(y_test,test_pred_sin)

0.7834853067745615